Implement a optimization method for feature combination / selection 

In [1]:
%load_ext autoreload
%autoreload 2

import pyVHR as vhr
import numpy as np
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.plot.visualize import *
import os
import plotly.express as px
from pyVHR.utils.errors import getErrors, printErrors, displayErrors

import constants
import pandas as pd
import pyVHR.analysis.pipelineLandmarks as custom_pipeline

vhr.plot.VisualizeParams.renderer = 'vscode' 

In [2]:
# -- LOAD A DATASET

dataset_name = 'lgi_ppgi'    
video_DIR, BVP_DIR = constants.get_dataset_paths(dataset_name)

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames
print("Number of videos: ", len(allvideo))  

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

GYM = np.arange(0,len(allvideo), 4)
RESTING = np.arange(1,len(allvideo), 4)
ROTATION = np.arange(2,len(allvideo), 4)
TALK = np.arange(3,len(allvideo), 4)

Number of videos:  24
0 D:/datasets_rppg/lgi_ppgi\alex\alex_gym\cv_camera_sensor_stream_handler.avi
1 D:/datasets_rppg/lgi_ppgi\alex\alex_resting\cv_camera_sensor_stream_handler.avi
2 D:/datasets_rppg/lgi_ppgi\alex\alex_rotation\cv_camera_sensor_stream_handler.avi
3 D:/datasets_rppg/lgi_ppgi\alex\alex_talk\cv_camera_sensor_stream_handler.avi
4 D:/datasets_rppg/lgi_ppgi\angelo\angelo_gym\cv_camera_sensor_stream_handler.avi
5 D:/datasets_rppg/lgi_ppgi\angelo\angelo_resting\cv_camera_sensor_stream_handler.avi
6 D:/datasets_rppg/lgi_ppgi\angelo\angelo_rotation\cv_camera_sensor_stream_handler.avi
7 D:/datasets_rppg/lgi_ppgi\angelo\angelo_talk\cv_camera_sensor_stream_handler.avi
8 D:/datasets_rppg/lgi_ppgi\cpi\cpi_gym\cv_camera_sensor_stream_handler.avi
9 D:/datasets_rppg/lgi_ppgi\cpi\cpi_resting\cv_camera_sensor_stream_handler.avi
10 D:/datasets_rppg/lgi_ppgi\cpi\cpi_rotation\cv_camera_sensor_stream_handler.avi
11 D:/datasets_rppg/lgi_ppgi\cpi\cpi_talk\cv_camera_sensor_stream_handler.avi
12

In [3]:
# -- PARAMETER SETTING

roi = 'mustache'      # jaw, forehead, cheeks, nose, temple, lip
wsize = 8        # seconds of video processed (with overlapping) for each estimate 
min_len = 2     # minimum number of landmarks tested
patch_size = constants.get_patch_size(dataset_name)
print(f"Patch size for {dataset_name} is {patch_size} ")
print(f"Testing ROI = {roi} with minimum {min_len} landmarks")  

Patch size for lgi_ppgi is 40 
Testing ROI = mustache with minimum 2 landmarks


# Hyperopt

https://celmore25.medium.com/automated-feature-selection-with-hyperopt-46865e1b4fce

Documentation: https://hyperopt.github.io/hyperopt/#documentation

Code: https://github.com/hyperopt/hyperopt

- It does not look like the fmin works too well when the numbe rof max iterations is less than the numbre of combinations. looks like ill have to do an exaustive search after all. 
- I can discard lips  and temple area

TODO change from run mlti methods to run dataset ? 


In [4]:
rois = {
  'forehead': [   
      'lower_medial_forehead','glabella','left_lower_lateral_forehead','right_lower_lateral_forehead'
    ],
 'nose': [
    'upper_nasal_dorsum','lower_nasal_dorsum','left_mid_nasal_sidewall','right_mid_nasal_sidewall','left_lower_nasal_sidewall',
    'right_lower_nasal_sidewall','nasal_tip','soft_triangle','left_ala','right_ala'
  ],
  'cheeks':[
    'left_malar','right_malar', 'left_lower_cheek','right_lower_cheek'
  ],
  'jaw':[
    'left_marionette_fold','right_marionette_fold','chin'
  ],
  'temple':[
    'left_temporal','right_temporal'
  ],
  'mustache':[
    'left_nasolabial_fold','right_nasolabial_fold','left_upper_lip','right_upper_lip','philtrum'
  ],
}

forehead_params = [['left_lower_lateral_forehead', 'right_lower_lateral_forehead'], ['glabella', 'lower_medial_forehead'],  ['left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead']]
cheeks_params =  [['left_malar', 'right_malar'], ['left_lower_cheek', 'right_lower_cheek'], ['left_lower_cheek', 'left_malar', 'right_lower_cheek', 'right_malar']]
jaw_params = [['left_marionette_fold', 'right_marionette_fold'], ['chin', 'left_marionette_fold', 'right_marionette_fold']]
mustache_params = [['left_nasolabial_fold', 'right_nasolabial_fold'], ['left_upper_lip', 'right_upper_lip'], ['left_nasolabial_fold', 'philtrum', 'right_nasolabial_fold'], ['left_upper_lip', 'philtrum', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'right_nasolabial_fold', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'philtrum', 'right_nasolabial_fold', 'right_upper_lip']]

In [6]:
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, hp, space_eval, FMinIter
from hyperopt_edit import refresh, run
from hyperopt.base import JOB_STATE_DONE
from hyperopt.early_stop import no_progress_loss
from itertools import chain, combinations

def get_combinations(elements, min_len=2):
    combs = list(chain.from_iterable(combinations(elements, r) for r in range(min_len, len(elements) + 1)))
    return [list(i) for i in combs]

# parameter space
parameter_space = {}
ldmks_list = rois[roi]

# ldmks_list = ['chin', 'glabella']
for ldmk in ldmks_list:
    parameter_space[ldmk] = hp.choice(ldmk, [0, 1])
print(f"Selection roi as {roi} with landmarks {ldmks_list}")
print(f"Number of possible combinations with {len(ldmks_list)} landmarks is :", 2**len(ldmks_list) - 1)

all_combinations = get_combinations(parameter_space.keys(), min_len)
for elem in all_combinations:
    elem.sort()
print(f"For min length {min_len}, number of all combinations is {len(all_combinations)}")
print(all_combinations)

valid_combinations = []
for comb in all_combinations:
    valid = True
    if 'left' in ''.join(comb):
        lefts = [s for s in comb if 'left' in s]
        for elem in lefts:
            if '_'.join(['right']+elem.split('_')[1:]) not in comb:
                valid = False
                break
    if 'right' in ''.join(comb):
        rights = [s for s in comb if 'right' in s]
        for elem in rights:
            if '_'.join(['left']+elem.split('_')[1:]) not in comb:
                valid = False
                break
    if valid:
        valid_combinations.append(comb)
print(f"For min length {min_len}, number of valid combinations is {len(valid_combinations)}")
print(valid_combinations)

Selection roi as mustache with landmarks ['left_nasolabial_fold', 'right_nasolabial_fold', 'left_upper_lip', 'right_upper_lip', 'philtrum']
Number of possible combinations with 5 landmarks is : 31
For min length 2, number of all combinations is 26
[['left_nasolabial_fold', 'right_nasolabial_fold'], ['left_nasolabial_fold', 'left_upper_lip'], ['left_nasolabial_fold', 'right_upper_lip'], ['left_nasolabial_fold', 'philtrum'], ['left_upper_lip', 'right_nasolabial_fold'], ['right_nasolabial_fold', 'right_upper_lip'], ['philtrum', 'right_nasolabial_fold'], ['left_upper_lip', 'right_upper_lip'], ['left_upper_lip', 'philtrum'], ['philtrum', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'right_nasolabial_fold'], ['left_nasolabial_fold', 'right_nasolabial_fold', 'right_upper_lip'], ['left_nasolabial_fold', 'philtrum', 'right_nasolabial_fold'], ['left_nasolabial_fold', 'left_upper_lip', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'philtrum'], ['left_nasolabial_

In [7]:
from hyperopt.early_stop import no_progress_loss

class HpOptBinarySelect:
    def __init__(self, videos, dataset, winsize, patch_size, space, min_len, pipeline, methods=['cupy_CHROM'], verb=False):
        self.dataset = dataset
        self.winsize = winsize
        self.patch_size = patch_size
        self.parameter_space = space
        self.methods = methods
        self.pipeline = pipeline
        self.res = pd.DataFrame()
        self.all_combinations = get_combinations(space.keys(), min_len)
        self.verb = verb

        # Load ground truth data
        self.sigGT = []
        self.timesGT = []
        self.bpmGT = []
        self.videoFileName = []
        self.load_ground_truth(videos)

    def get_combinations(elements, min_len=2):
        combs = list(chain.from_iterable(combinations(elements, r) for r in range(min_len, len(elements) + 1)))
        return [list(i) for i in combs]

    def load_ground_truth(self, videos):
        for videoIdx in videos:
            try: 
                fname = dataset.getSigFilename(videoIdx)
                sigGT = dataset.readSigfile(fname)
                bpmGT, timesGT = sigGT.getBPM(self.winsize)
                self.sigGT.append(sigGT)
                self.bpmGT.append(bpmGT)
                self.timesGT.append(timesGT)
                self.videoFileName.append(dataset.getVideoFilename(videoIdx))
                self.fps = vhr.extraction.get_fps(self.videoFileName[-1]) # assuming they are all the same
            except Exception as e:
                print(f"{videoIdx}: {e}")
                continue
        print('Video name: ', {len(self.videoFileName)}, self.videoFileName)
        print('Video frame rate: ',self.fps)

    def objective(self,params):
        ldmks_list = [i for i,j in params.items() if j==1]
        print(f"LANDMARKS: {ldmks_list}")

        # TODO maybe i should just define params as such
        if ldmks_list not in valid_combinations:
            return {'loss': np.nan,'status': STATUS_FAIL}
        self.all_combinations = valid_combinations

        # Less than 2 landmarks tested
        if len(ldmks_list) < min([len(combs) for combs in self.all_combinations]):
            return {'loss': np.nan,'status': STATUS_FAIL}
    
        # Get past landmarks that were tested
        past_param_values = [trial['misc']['vals'] for trial in self.trials._dynamic_trials if trial['result']['status'] == 'ok']
        past_landmarks = []
        for value in past_param_values:
            past_landmarks.append([i for i,j in value.items() if j[0]==1])

        # All combinations have been tested
        if len(past_param_values) == len(self.all_combinations):
            print(f"All {len(past_param_values)} combinations have been tested ")
            self.trials._fmin_cancelled = True
            return {'loss': np.nan,'status': STATUS_FAIL}

        # Remove duplicate trials
        if ldmks_list in past_landmarks:
            return {'loss': np.nan,'status': STATUS_FAIL}
        
        losses = []
        for i, videoName in enumerate(self.videoFileName):
            print(videoName)
            res = self.pipeline.run_on_video_multimethods(
                    ldmks_list=ldmks_list, 
                    videoFileName=self.videoFileName[i], bpmGT=self.bpmGT[i], timesGT=self.timesGT[i], 
                    methods=self.methods, winsize=self.winsize, patch_size=self.patch_size,
                    verb=self.verb
                )
            losses.append(res.dict['RMSE'][0]) # suppose we are minimizing RMSE
            self.res = pd.concat([self.res, res.dataFrame])
        print(f"Total loss for {ldmks_list}: {sum(losses)}")
        return {'loss': sum(losses), 'status': STATUS_OK}

    def optimize(self, max_evals=20):
        self.trials = Trials()
        best = fmin(fn=self.objective,
                    space=self.parameter_space,
                    algo=tpe.suggest,
                    max_evals=max_evals,
                    trials=self.trials,
                    # early_stop_fn=no_progress_loss(10),
                    )
        return space_eval(self.parameter_space, best)
    
FMinIter.run = run
Trials.refresh = refresh

In [9]:
# np.arange(0, len(dataset.videoFilenames))
pl = custom_pipeline.LandmarksPipeline()
hpobj = HpOptBinarySelect(
    videos=np.arange(0, len(dataset.videoFilenames)), dataset=dataset, winsize=8, patch_size=patch_size, 
    space=parameter_space, min_len=2, pipeline=pl, methods=['cupy_CHROM'], verb=False
)

best = hpobj.optimize(max_evals=20)
out = [i for i,j in best.items() if j==1]
print(f'Final number of features {len(out)}: ', out)

4: Unusable data.
16: Unusable data.
Video name:  {22} ['D:/datasets_rppg/lgi_ppgi\\alex\\alex_gym\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\alex\\alex_resting\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\alex\\alex_rotation\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\alex\\alex_talk\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\angelo\\angelo_resting\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\angelo\\angelo_rotation\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\angelo\\angelo_talk\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\cpi\\cpi_gym\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\cpi\\cpi_resting\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\cpi\\cpi_rotation\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\cpi\\cpi_talk\\cv_camera_sensor_stream_handler.avi', 'D:/data

In [20]:
df = hpobj.res.reset_index(drop=True)
if dataset_name == 'lgi_ppgi':
  df['videoFilename'] = df.videoFilename.apply(lambda x: x.split('\\')[2])
if dataset_name == 'mr_nirp':
  df['videoFilename'] = df.videoFilename.apply(lambda x: x.split('\\')[2])
indexes = {}
for v in range(len(allvideo)):
  indexes[allvideo[v].split('\\')[2]] = v
indexes = pd.DataFrame({'videoIdx':list(indexes.values()), 'videoFilename':list(indexes.keys())})
df = df.drop(columns=['videoIdx'])
df.insert(3, 'videoIdx', df['videoFilename'].map(indexes.set_index('videoFilename')['videoIdx']))

from fastdtw import fastdtw
df.insert(11, 'DTW', None)
for row in df.itertuples():
  distance, path = fastdtw(row.bpmGT, row.bpmES)
  df.loc[row.Index, 'DTW'] = [distance]

print("Number of unique combinations tested: ", df.landmarks.apply(lambda x: tuple(x)).unique().size)
print("Best score achieved with : ", out)
print(f"Number of NaN rows {df['bpmGT'].isna().sum()} ({df[df['bpmGT'].isna()]['videoFilename'].iloc[0] if df['bpmGT'].isna().sum().sum() > 0 else None})")
df[df['landmarks'].apply(lambda x: x == out)][['videoFilename', 'RMSE' ,'MAE', 'PCC']]

# df['landmarks'] = df['landmarks'].apply(lambda x: tuple(x))
# df['PCC'] = df['PCC'].apply(lambda x: abs(x[0])) 
# df[['landmarks', 'videoIdx', 'MAE', 'RMSE', 'PCC']].groupby('landmarks').mean().sort_values(by='RMSE')

Number of unique combinations tested:  21
Best score achieved with :  ['left_lower_nasal_sidewall', 'right_lower_nasal_sidewall', 'soft_triangle', 'upper_nasal_dorsum']
Number of NaN rows 0 (None)


,videoFilename,RMSE,MAE,PCC
30,alex_rotation,[2.99378602950556],[1.8187529151119404],[0.5968948477068808]
31,angelo_rotation,[1.1282260722329789],[0.9119073275862067],[0.9015606547823574]
32,cpi_rotation,[5.140586255986128],[1.9761745505136987],[0.43310261673201217]
33,david_rotation,[5.190569068566008],[3.0767578125],[0.45177221706915804]
34,felix_rotation,[6.271504792249803],[5.425162760416667],[0.3535733204512786]
35,harun_rotation,[14.911925309480743],[9.184849330357142],[0.1383186417951882]


In [22]:
df.head(1)

,method,dataset,videoIdx,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,DTW,MAD,bpmGT,bpmES,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,cupy_CHROM,lgi_ppgi_nose,2,alex_rotation,[4.5748339907963755],[2.902664412313433],[0.47144104490323235],[0.3167227385864613],[-0.5598206185247513],[18.2939453125],[149.1689453125],"[1.46484375, 12.8173828125, 14.2822265625, 17....","[67.0, 67.5, 68.5, 69.0, 69.5, 70.0, 69.5, 68....","[68.84765625, 67.0166015625, 67.3828125, 65.91...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",39.545573,"(left_lower_nasal_sidewall, left_mid_nasal_sid..."


In [21]:
df['landmarks'] = df['landmarks'].apply(lambda x: tuple(x))
df = df.drop(columns=['sigFilename', 'bpmES_mad'])
df['dataset'] = f'{dataset_name}_{roi}'

print("saving into ", f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5')
df.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_rotation_{roi}.h5', key='df', mode='w')
# df_old = pd.read_hdf(f'../results/test_landmarks/h5/{dataset_name.upper()}/{dataset_name.upper()}_{roi}.h5', key='df')
# df_new = pd.concat([df_old, df], ignore_index=True)
# df_new.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5', key='df', mode='w')

saving into  ../results/test_landmarks/LGI_PPGI_nose.h5


C:\Users\erolland\AppData\Local\Temp\3\ipykernel_22476\707446877.py:6: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['method', 'dataset', 'videoFilename', 'RMSE', 'MAE', 'PCC', 'CCC',
       'SNR', 'MAX', 'DTW', 'MAD', 'bpmGT', 'bpmES', 'timeGT', 'timeES',
       'landmarks'],
      dtype='object')]

  df.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_rotation_{roi}.h5', key='df', mode='w')


In [48]:
x = pd.read_hdf(f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5', key='df')
# x = pd.read_hdf(f'../results/test_landmarks/h5/{dataset_name.upper()}/{dataset_name.upper()}_{roi}.h5', key='df')
print(x.videoIdx.unique())
print(x.landmarks.unique().size)

[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 23]
5


In [49]:
x.head(2)

,method,dataset,videoIdx,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,DTW,MAD,bpmGT,bpmES,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,cupy_CHROM,lgi_ppgi_forehead,0,alex_gym,[24.665734276197384],[14.951978165064103],[0.556308586949296],[0.519186543628614],[-4.341552955811867],[92.330078125],[5114.220703125],"[1.0986328125, 15.380859375, 4.7607421875, 8.0...","[101.0, 100.0, 97.0, 94.5, 93.0, 93.0, 93.0, 9...","[56.0302734375, 71.044921875, 86.7919921875, 6...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",207.187744,"(left_lower_lateral_forehead, right_lower_late..."
1,cupy_CHROM,lgi_ppgi_forehead,1,alex_resting,[1.4564609910056892],[0.948449337121212],[0.9288874300939538],[0.9252186060984902],[2.4762156521407603],[5.65234375],[45.38281250000001],"[0.732421875, 0.732421875, 0.3662109375, 0.732...","[68.5, 68.5, 68.0, 67.0, 67.0, 67.0, 67.0, 66....","[67.0166015625, 67.3828125, 67.0166015625, 66....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",37.899232,"(left_lower_lateral_forehead, right_lower_late..."


In [50]:
x[['landmarks', 'videoIdx','videoFilename', 'MAE', 'RMSE', 'PCC']].sort_values(['landmarks', 'videoIdx'])

,landmarks,videoIdx,videoFilename,MAE,RMSE,PCC
44,"(glabella, left_lower_lateral_forehead, lower_...",0,alex_gym,[12.999063501602564],[22.821534677644447],[0.6190955904114843]
45,"(glabella, left_lower_lateral_forehead, lower_...",1,alex_resting,[0.959191524621212],[1.3920077137735014],[0.9362377476863841]
46,"(glabella, left_lower_lateral_forehead, lower_...",2,alex_rotation,[4.497493003731344],[7.780106122289471],[0.37194563237620104]
47,"(glabella, left_lower_lateral_forehead, lower_...",3,alex_talk,[5.612781613372093],[10.174042255221346],[-0.01571755782221277]
48,"(glabella, left_lower_lateral_forehead, lower_...",5,angelo_resting,[0.7071967690677967],[0.8799477042366245],[0.9556039339132313]
...,...,...,...,...,...,...
17,"(left_lower_lateral_forehead, right_lower_late...",19,felix_talk,[12.99295291385135],[17.668801110735227],[0.0556875811431317]
18,"(left_lower_lateral_forehead, right_lower_late...",20,harun_gym,[30.98845223063973],[41.0358402992574],[0.011294375701483292]
19,"(left_lower_lateral_forehead, right_lower_late...",21,harun_resting,[1.3983917236328125],[2.8444196284687964],[0.5770411619400944]
20,"(left_lower_lateral_forehead, right_lower_late...",22,harun_rotation,[12.371279761904763],[17.95543092549593],[0.34906218614705675]


# AAAH FIX

In [22]:
# x = pd.read_hdf(f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5', key='df')
x = pd.read_hdf(f'../results/test_landmarks/h5/{dataset_name.upper()}/{dataset_name.upper()}_{roi}.h5', key='df')
x = x.sort_values("videoFilename").reset_index(drop=True)
print(x.shape)
print(len(x.videoIdx.unique()), x.videoIdx.unique())
print(x.landmarks.unique().size)

(66, 16)
22 [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 23]
3


In [23]:
sigGT_list = []
bpmGT_list = []
timesGT_list = []
videoFileName = []
winsize = 8
videos = np.arange(0, len(dataset.videoFilenames))

for videoIdx in videos:
    try:
        fname = dataset.getSigFilename(videoIdx)
        print(fname)
        sigGT = dataset.readSigfile(fname)
        bpmGT, timesGT = sigGT.getBPM(winsize)
        sigGT_list.append(sigGT)
        bpmGT_list.append(bpmGT)
        timesGT_list.append(timesGT)
        videoFileName.append(dataset.getVideoFilename(videoIdx))
        fps = vhr.extraction.get_fps(videoFileName[-1]) # assuming they are all the same
    except:
        continue
print('Video name: ', {len(videoFileName)}, videoFileName)
print('Video frame rate: ',fps)

D:/datasets_rppg/lgi_ppgi\alex\alex_gym\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\alex\alex_resting\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\alex\alex_rotation\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\alex\alex_talk\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\angelo\angelo_gym\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\angelo\angelo_resting\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\angelo\angelo_rotation\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\angelo\angelo_talk\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\cpi\cpi_gym\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\cpi\cpi_resting\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\cpi\cpi_rotation\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\cpi\cpi_talk\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\david\david_gym\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\david\david_resting\cms50_stream_handler.xml
D:/datasets_rppg/lgi_ppgi\david\david_rotati

In [30]:
x.iloc[4:7]

,method,dataset,videoIdx,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,MAD,bpmGT,bpmES,timeGT,timeES,landmarks
4,cupy_CHROM,LGI_PPGI_chin,1,alex_resting,1.8371726116174953,1.0355557528409092,0.8783785342344415,0.8711894918445292,nan,9.779296875,"[0.732421875, 1.46484375, 0.3662109375, 0.7324...","[68.5, 68.5, 68.0, 67.0, 67.0, 67.0, 67.0, 66....","[66.650390625, 67.3828125, 67.0166015625, 67.3...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....","(left_marionette_fold, right_marionette_fold)"
5,cupy_CHROM,LGI_PPGI_chin,1,alex_resting,3.027574319272114,1.6256066524621209,0.7216510924472809,0.7152519426821968,nan,12.3427734375,"[0.732421875, 0.732421875, 1.0986328125, 1.464...","[68.5, 68.5, 68.0, 67.0, 67.0, 67.0, 67.0, 66....","[66.650390625, 68.115234375, 68.4814453125, 67...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....","(chin,)"
6,cupy_CHROM,LGI_PPGI_chin,2,alex_rotation,11.542181375333193,7.149341184701493,0.3976674507570197,0.12272494829540258,nan,53.1103515625,"[1.0986328125, 5.4931640625, 9.8876953125, 6.2...","[67.0, 67.5, 68.5, 69.0, 69.5, 70.0, 69.5, 68....","[70.6787109375, 70.3125, 61.5234375, 57.128906...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....","(chin, left_marionette_fold, right_marionette_..."


In [28]:
x.iloc[4:7]

,method,dataset,videoIdx,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,MAD,bpmGT,bpmES,timeGT,timeES,landmarks
4,cupy_CHROM,LGI_PPGI_chin,1,alex_resting,1.8371726116174953,1.0355557528409092,0.8783785342344415,0.8711894918445292,nan,9.779296875,"[0.732421875, 1.46484375, 0.3662109375, 0.7324...","[68.5, 68.5, 68.0, 67.0, 67.0, 67.0, 67.0, 66....","[66.650390625, 67.3828125, 67.0166015625, 67.3...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....","(left_marionette_fold, right_marionette_fold)"
5,cupy_CHROM,LGI_PPGI_chin,1,alex_resting,3.027574319272114,1.6256066524621209,0.7216510924472809,0.7152519426821968,nan,12.3427734375,"[0.732421875, 0.732421875, 1.0986328125, 1.464...","[68.5, 68.5, 68.0, 67.0, 67.0, 67.0, 67.0, 66....","[66.650390625, 68.115234375, 68.4814453125, 67...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....","(chin,)"
6,cupy_CHROM,LGI_PPGI_chin,2,alex_rotation,11.542181375333193,7.149341184701493,0.3976674507570197,0.12272494829540258,nan,53.1103515625,"[1.0986328125, 5.4931640625, 9.8876953125, 6.2...","[67.0, 67.5, 68.5, 69.0, 69.5, 70.0, 69.5, 68....","[70.6787109375, 70.3125, 61.5234375, 57.128906...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....","(chin, left_marionette_fold, right_marionette_..."


In [27]:
bpmES = list(x.bpmES)
timesES = list(x.timeES)
nb = x.landmarks.unique().size # nb landmarks
print(nb)
print(len(bpmES), len(bpmGT_list), len(timesES), len(timesGT_list))
for i in range(len(bpmES)):
    try:
        RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(None, fps, bpmES[i], bpmGT_list[i//nb], timesES[i], timesGT_list[i//nb])
    except:
        RMSE, MAE, MAX, PCC, CCC, SNR, MAD = [np.nan], [np.nan], [np.nan], [np.nan], [np.nan], [np.nan], [np.nan]
        bpmGT, bpmES, timesGT, timesES = np.nan, np.nan, np.nan, np.nan
    x.loc[i,'RMSE'] = RMSE
    x.loc[i,'MAE'] = MAE
    x.loc[i,'MAX'] = MAX
    x.loc[i,'PCC'] = PCC
    x.loc[i,'CCC'] = CCC
    x.loc[i,'SNR'] = SNR
    x.at[i,'bpmGT'] = bpmGT_list[i//nb]
    x.at[i,'timeGT'] = timesGT_list[i//nb]   

3
66 22 66 22


# Test Hyperopt

In [15]:
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, hp, space_eval, FMinIter
from hyperopt_edit import refresh, run

FMinIter.run = run
Trials.refresh = refresh
trials = Trials()

def func(x):
    print(f"RUN {x}")
    past_param_values = [trial['misc']['vals'] for trial in trials._dynamic_trials if trial['result']['status'] == 'ok']
    print(f"Past landmarks: {past_param_values}")
    if len(past_param_values) >= 3:
        print(f"All combinations have been tested {len(past_param_values)}")
        print(f"first one should be safe {trials.trials[0]} ")
        if hasattr(trials.trials, "_fmin_cancelled"):
            print(f"Cancelling trials {trials._fmin_cancelled}")
        trials._fmin_cancelled = True
        return {"loss":np.nan,"status": STATUS_FAIL}
    
    if (x < 0):
        return {"loss": (x - 3) ** 2, "status": STATUS_OK}  
    else:
        print(f"ERROR ERROR ERROR {len(trials._dynamic_trials)}")
        return {"loss":np.nan,"status": STATUS_FAIL}

argmin = fmin(
    fn=func,
    space=hp.uniform("x", -1, 2),
    algo=tpe.suggest,
    max_evals=5,
    trials=trials,
    catch_eval_exceptions = False,
)

print(argmin)

                                                     
     while 2: True, False
     while 3 qlen, self.max_queue_len, n_queued, N: 0, 1, 0, 5, True, True, True
     n_queued: 0 q_len 1                             
RUN 1.5087915254710804                               
Past landmarks: []                                   
ERROR ERROR ERROR 1                                  
     while 2.1 unfinished, done, done this iteration: 0 0 0
     while 2.2: 0 5, True False False                
                                                     
     while 2: True, False
     while 3 qlen, self.max_queue_len, n_queued, N: 0, 1, 0, 5, True, True, True
     n_queued: 0 q_len 1                             
RUN 1.809393828244759                                
Past landmarks: []                                   
ERROR ERROR ERROR 2                                  
     while 2.1 unfinished, done, done this iteration: 0 0 0
     while 2.2: 0 5, True False False                
                  

In [16]:
print("Number of trials: ", len(trials._dynamic_trials), len(trials.trials))

Number of trials:  8 3
